# Quiz 1

In [1]:
pip install pyspark

# Part 1

## Task 1: Count the number of words starting with a specific letter.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("word_count").getOrCreate()

# Read the text file into a DataFrame
text_df = spark.read.csv("SPAM SMS.csv")
print(text_df)
# Split the lines into words
words_df = text_df.selectExpr("explode(split(_c0, ' ')) as word")
print(words_df)
# Count the number of words starting with a specific letter (e.g., 'n')
letter_count = words_df.filter(col("word").startswith("h")).count()

print(f"Number of words starting with 'h': {letter_count}")

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]
DataFrame[word: string]
Number of words starting with 'h': 4827


## Task 2: Count the number of occurrences of a specific word.

In [3]:
# Specify the word to count
target_word ="ham"

# Count the occurrences of the specific word
word_count = words_df.filter(col("word") == target_word).count()

print(f"Occurrences of '{target_word}': {word_count}")


Occurrences of 'ham': 4825


## Task 3: Convert all words to uppercase and count the occurrences.

In [4]:

# Convert all words to uppercase
uppercase_words_df = words_df.selectExpr("upper(word) as word")

# Count the occurrences of each word
uppercase_word_counts = uppercase_words_df.groupBy("word").count()

uppercase_word_counts.show()

+------+-----+
|  word|count|
+------+-----+
|  SPAM|  747|
|HAM"""|    2|
|   HAM| 4825|
|    V1|    1|
+------+-----+



## Task 4: Filter out words of length less than 5 and count the occurrences.

In [5]:
from pyspark.sql.functions import length

# Filter out words of length less than 5
filtered_words_df = words_df.filter(length("word") >= 5)

# Count the occurrences of each filtered word
filtered_word_counts = filtered_words_df.groupBy("word").count()

filtered_word_counts.show()

+------+-----+
|  word|count|
+------+-----+
|ham"""|    2|
+------+-----+



## Task 5: Count the number of lines containing a specific word.

In [6]:
# Specify the word to check in lines
word_in_lines = "spam"

# Specify the correct column name containing the text
text_column_name = "_c0"

# Count the number of lines containing the specific word
lines_with_word_count = text_df.filter(col(text_column_name).contains(word_in_lines)).count()

print(f"Number of lines containing '{word_in_lines}': {lines_with_word_count}")


Number of lines containing 'spam': 747


# Part 2

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.streaming import StreamingQuery

# Create a Spark Session
spark = SparkSession.builder.appName('Person Data').getOrCreate()

# Create a sample dataset
data = [("John", "Doe", 22),
        ("Jane", "Doe", 28),
        ("Bob", "Johnson", 30),
        ("Alice", "Smith", 21),
        ("James", "Johnson", 26)]

columns = ['FirstName','LastName','Age']

df= spark.createDataFrame(data,columns)

# Task 1: Select people younger than 25 years old
young_people =df.filter(col("Age")< 25)

# Task 2: Find the average age of people whose last name starts with the letter 'D'
average_age_D = df.filter(col("LastName").startswith("D")).agg(F.avg("Age").alias("AverageAge"))

# Task 3: Get the dataset's total count of distinct last names
distinct_last_names_count = df.select("LastName").distinct().count()

# Task 4: Find the maximum age of people whose first name starts with the letter 'J'
max_age_J = df.filter(col("FirstName").startswith("J")).agg(F.max("Age").alias("MaxAge"))

# Display results
print("Young People:")
young_people.show()

print("Average Age of People with Last Name starting with 'D':")
average_age_D.show()

print("Total Count of Distinct Last Names:")
print(distinct_last_names_count)

print("Maximum Age of People with First Name starting with 'J':")
max_age_J.show()

Young People:
+---------+--------+---+
|FirstName|LastName|Age|
+---------+--------+---+
|     John|     Doe| 22|
|    Alice|   Smith| 21|
+---------+--------+---+

Average Age of People with Last Name starting with 'D':
+----------+
|AverageAge|
+----------+
|      25.0|
+----------+

Total Count of Distinct Last Names:
3
Maximum Age of People with First Name starting with 'J':
+------+
|MaxAge|
+------+
|    28|
+------+



# Part 3

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Create a DataFrame with sample data
data = [("John", "Smith", 25),
        ("Jane", "Johnson", 30),
        ("Alice", "Jones", 22),
        ("Jack", "Jackson", 28),
        ("Jerry", "Johnson", 25),
        # Add more data as needed
        ]

columns = ["first_name", "last_name", "age"]

df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

# Task 1: Get the sum of ages of people whose last name contains 's'
sum_ages_with_s = df.filter(col("last_name").like("%s%")).agg({"age": "sum"}).collect()[0][0]
print(f"Sum of ages of people whose last name contains 's': {sum_ages_with_s}")

# Task 2: Get the number of distinct first names that start with 'J' and end with 'e'
distinct_j_first_names = df.filter((col("first_name").startswith("J")) & (col("first_name").endswith("e"))).select("first_name").distinct().count()
print(f"Number of distinct first names starting with 'J' and ending with 'e': {distinct_j_first_names}")

# Task 3: Get the number of distinct last names that start with 'J' and end with 'n'
distinct_j_last_names = df.filter((col("last_name").startswith("J")) & (col("last_name").endswith("n"))).select("last_name").distinct().count()
print(f"Number of distinct last names starting with 'J' and ending with 'n': {distinct_j_last_names}")

# Task 4: Get the first and last names of the person aged 25
names_of_age_25 = df.filter(col("age") == 25).select("first_name", "last_name").collect()
print(f"First and last names of people aged 25: {names_of_age_25}")

# You can also use SQL queries with the DataFrame
df.createOrReplaceTempView("people")

# SQL query example:
result = spark.sql("SELECT * FROM people WHERE age = 25")
result.show()

# Stop the Spark session
spark.stop()


+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|      John|    Smith| 25|
|      Jane|  Johnson| 30|
|     Alice|    Jones| 22|
|      Jack|  Jackson| 28|
|     Jerry|  Johnson| 25|
+----------+---------+---+

Sum of ages of people whose last name contains 's': 105
Number of distinct first names starting with 'J' and ending with 'e': 1
Number of distinct last names starting with 'J' and ending with 'n': 2
First and last names of people aged 25: [Row(first_name='John', last_name='Smith'), Row(first_name='Jerry', last_name='Johnson')]
+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|      John|    Smith| 25|
|     Jerry|  Johnson| 25|
+----------+---------+---+



# part 4

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Define the schema for the DataFrame
schema = StructType([
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("age", IntegerType(), True),
])

# Sample data
data = [
    ("Alex", "Smith", 40),
    ("Jane", "Doe", 30),
    ("John", "Doe", 35),
    ("Mike", "Johnston", 25),
]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)

# Show the original DataFrame
print("Original DataFrame:")
df.show()

# Update the age of 'Alex Smith' to 41
df = df.withColumn("age", when((col("first_name") == "Alex") & (col("last_name") == "Smith"), 41).otherwise(col("age")))

# Update the first name of 'Jane Doe' to 'Janet'
df = df.withColumn("first_name", when((col("first_name") == "Jane") & (col("last_name") == "Doe"), "Janet").otherwise(col("first_name")))

# Delete 'John Doe' from the table
df = df.filter((col("first_name") != "John") & (col("last_name") != "Doe"))

# Delete 'Mike Johnston' from the table
df = df.filter((col("first_name") != "Mike") & (col("last_name") != "Johnston"))

# Show the updated DataFrame
print("\nUpdated DataFrame:")
df.show()


Original DataFrame:
+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|      Alex|    Smith| 40|
|      Jane|      Doe| 30|
|      John|      Doe| 35|
|      Mike| Johnston| 25|
+----------+---------+---+


Updated DataFrame:
+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|      Alex|    Smith| 41|
+----------+---------+---+

